In [275]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


def get_track_info(track_name):
    # Set up Spotipy client with your Spotify API credentials
    client_credentials_manager = SpotifyClientCredentials(client_id='89357b1de85a4b758d8bcaed6974a7c5',
                                                          client_secret='d764e749f046434ead473dcd01fb116e')
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    # Search for the track
    results = sp.search(q=track_name, type='track', limit=1)
    if not results['tracks']['items']:
        return 'Track not found'

    track = results['tracks']['items'][0]

    # Retrieve audio features of the track
    audio_features = sp.audio_features(track['id'])[0]

    # Retrieve artist information
    artist = sp.artist(track['artists'][0]['id'])

    # Compile the requested information
    track_info = {
        'year': track['album']['release_date'][:4],
        'track_name': track['name'],
        'track_popularity': track['popularity'],
        'album': track['album']['name'],
        'artist_name': track['artists'][0]['name'],
        'artist_genres': artist['genres'],
        'artist_popularity': artist['popularity'],
        'danceability': audio_features['danceability'],
        'energy': audio_features['energy'],
        'key': audio_features['key'],
        'loudness': audio_features['loudness'],
        'mode': audio_features['mode'],
        'speechiness': audio_features['speechiness'],
        'acousticness': audio_features['acousticness'],
        'instrumentalness': audio_features['instrumentalness'],
        'liveness': audio_features['liveness'],
        'valence': audio_features['valence'],
        'tempo': audio_features['tempo'],
        'duration_ms': audio_features['duration_ms'],
    }

    return track_info

# Example usage
track_name = input('Please provide me a track name:')
track_info = get_track_info(track_name=track_name)
print(track_info)

{'year': '2018', 'track_name': 'Füttyös', 'track_popularity': 46, 'album': 'Füttyös', 'artist_name': 'Majka', 'artist_genres': ['hungarian edm', 'hungarian pop'], 'artist_popularity': 43, 'danceability': 0.434, 'energy': 0.856, 'key': 8, 'loudness': -4.761, 'mode': 0, 'speechiness': 0.274, 'acousticness': 0.481, 'instrumentalness': 0, 'liveness': 0.4, 'valence': 0.736, 'tempo': 99.175, 'duration_ms': 212000}


In [276]:
type(track_info)

dict

In [277]:
# Example usage
track_info_df = pd.DataFrame(track_info)
track_info_df


,year,track_name,track_popularity,album,artist_name,artist_genres,artist_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,2018,Füttyös,46,Füttyös,Majka,hungarian edm,43,0.434,0.856,8,-4.761,0,0.274,0.481,0,0.4,0.736,99.175,212000
1,2018,Füttyös,46,Füttyös,Majka,hungarian pop,43,0.434,0.856,8,-4.761,0,0.274,0.481,0,0.4,0.736,99.175,212000


# Preprocessing

In [278]:
import re
import pandas as pd
import numpy as np

# internal
from joblib import load

In [279]:
import re
sklearn_pca_final = load('PCAmodel_audiofeatures.joblib')

mapping = {
    "pop": ['pop', 'pop punk', 'dance pop', 'europop', 'girl group', 'pop rap', 'bahamian pop', 'canadian pop', 'pop dance', 'post-teen pop', 'boy band', 'art pop', 'metropopolis', 'barbadian pop', 'viral pop', 'candy pop', 'australian pop', 'folk-pop', 'k-pop', 'bubblegum dance', 'pop dance'],
    "rock": ['permanent wave', 'alternative metal', 'modern rock', 'punk', 'rock', 'socal pop punk', 'alternative rock', 'dance rock', 'pop rock', 'celtic rock', 'irish rock', 'grunge', 'uk pop', 'neo mellow', 'piano rock', 'pov: indie', 'modern alternative rock', 'neon pop punk', 'garage rock', 'indie rock', 'canadian rock', 'british invasion', 'emo', 'neo-psychedelic', 'screamo', 'glam metal', 'beatlesque', 'madchester', 'supergroup'],
    "country": ['contemporary country', 'country', 'country dawn', 'country road'],
    "hip_hop_rap": ['detroit hip hop', 'hip hop', 'rap', 'dirty south rap', 'hip pop', 'east coast hip hop', 'hardcore hip hop', 'gangster rap', 'west coast rap', 'st louis rap', 'trap', 'southern hip hop', 'atl hip hop', 'melodic rap', 'miami hip hop', 'chicago rap', 'canadian hip hop', 'old school atlanta hip hop', 'queens hip hop', 'crunk', 'conscious hip hop', 'trap queen', 'south carolina hip hop', 'new orleans rap', 'G Funk', 'brooklyn drill', 'hyphy', 'atlanta bass'],
    "rnb_soul": ['contemporary r&b', 'r&b', 'urban contemporary', 'british soul', 'neo soul', 'pop soul', 'canadian contemporary r&b', 'soul', 'pluggnb'],
    "house_and_pop_fusion": ['disco house', 'filter house', 'bouncy house', 'electro house', 'slap house', 'tropical house', 'pop dance', 'house', 'progressive house', 'indietronica', 'progressive electro house', 'uk garage', 'big room', 'complextro'],
    "underground_electronic": ['electro', 'big beat', 'downtempo', 'melbourne bounce international', 'electro trash', 'grave wave', 'dark clubbing', 'ukg revival', 'hardcore techno', 'grime', 'instrumental grime', 'dancefloor dnb', 'trance', 'speed garage', 'new wave', 'bounce', 'handsup', 'chicago hardcore', 'rave', 'german techno', 'hamburg electronic', 'happy hardcore', 'industrial', 'techno'],
    "metal": ['nu metal', 'post-grunge', 'rap metal', 'funk metal', 'industrial metal'],
    "jazz": ['jazz', 'smooth jazz', 'bebop', 'swing', 'fusion jazz', 'stomp and holler'],
    "classical": ['classical', 'baroque', 'romantic', 'symphony', 'opera'],
    "reggae": ['reggae', 'dancehall', 'reggae fusion', 'dub', 'roots reggae', 'moombahton'],
    "latin": ['latin', 'reggaeton', 'latin pop', 'salsa', 'bachata', 'merengue', 'tango', 'cumbia', 'trap latino', 'urbano latino', 'mexican pop', 'colombian pop', 'mambo chileno', 'urbano chileno', 'tropical'],
    "singer-songwriter": ['singer-songwriter pop', 'singer-songwriter'],
    "folk": ['folk', 'classic schlager', 'oktoberfest', 'schlager', 'celtic', 'middle earth', 'sertanejo', 'sertanejo universitario'],
    "talent show": ['talent show']
}

# Flatten the mapping for easier access
flat_mapping = {}
for main_genre, subgenres in mapping.items():
    for subgenre in subgenres:
        flat_mapping[subgenre] = main_genre

# When a "new" sub genre is being added with a version of the artist_genre


def categorize_genres(genres_list, mapping):
    genre_counts = {}
    for genre in genres_list:
        for subgenre, main_genre in mapping.items():
            if re.search(r'\b' + re.escape(subgenre) + r'\b', genre, flags=re.IGNORECASE):
                genre_counts[main_genre] = genre_counts.get(main_genre, 0) + 1
    if genre_counts:
        # Return the most frequent main genre, if tie, sorted order decides
        return max(genre_counts, key=genre_counts.get)
    else:
        return "other"  # or "other" if you prefer to label unmatched genres as "other"

# Function to categorize genres


def categorize_genres(genres_list, mapping):
    genre_counts = {}
    for genre in genres_list:
        for subgenre, main_genre in mapping.items():
            if subgenre in genre:
                genre_counts[main_genre] = genre_counts.get(main_genre, 0) + 1
    if genre_counts:
        # Return the most frequent main genre, if tie, sorted order decides
        return max(genre_counts, key=genre_counts.get)
    else:
        return "other"  # or "other" if you prefer to label unmatched genres as "other"

In [280]:
def preprocess_data(track_data):
    # Convert track_data into a DataFrame
    tdata = pd.DataFrame(track_data)

    # Processing
    tdata['track_age_2024'] = 2024 - \
        pd.to_numeric(tdata['year'], errors='coerce')
    tdata['dominant_genre'] = tdata['artist_genres'].apply(
        lambda x: categorize_genres(x, flat_mapping))

    key_mapping = {0: 'C', 1: 'C#', 2: 'D', 3: 'D#', 4: 'E',
                   5: 'F', 6: 'F#', 7: 'G', 8: 'G#', 9: 'A', 10: 'A#', 11: 'B'}
    tdata['key'] = tdata['key'].map(key_mapping)
    mode_mapping = {0: 'min', 1: 'Maj'}
    tdata['mode'] = tdata['mode'].map(mode_mapping)
    tdata['tonality'] = tdata['key'] + tdata['mode']

    # One Hot Encoding for 'dominant_genre' and 'tonality'
    tdata = pd.get_dummies(tdata, columns=['dominant_genre', 'tonality'], prefix=[
                           'genre', 'tone'], drop_first=True)

    # Prepare and transform audio features with PCA
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness',
                      'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
    pca_components = sklearn_pca_final.transform(tdata[audio_features])
    pca_feature_names = ['PC1_Energetic_Dynamic', 'PC2_Speech_Tempo',
                         'PC3_Acoustic_Speech', 'PC4_Duration_Mood', 'PC5_Acoustic_Liveness']
    for i, name in enumerate(pca_feature_names):
        tdata[name] = pca_components[:, i]

    # Define expected features as per model's training
    model_features = ['genre_country', 'genre_folk', 'genre_hip_hop_rap', 'genre_house_and_pop_fusion',
                      'genre_jazz', 'genre_latin', 'genre_metal', 'genre_other', 
                      'genre_pop', 'genre_reggae', 'genre_rnb_soul', 'genre_rock', 
                      'genre_singer-songwriter', 'genre_underground_electronic', 
                      'tone_A#min', 'tone_AMaj', 'tone_Amin', 'tone_BMaj', 
                      'tone_Bmin', 'tone_C#Maj', 'tone_C#min', 'tone_CMaj', 
                      'tone_Cmin', 'tone_D#Maj', 'tone_D#min', 'tone_DMaj', 
                      'tone_Dmin', 'tone_EMaj', 'tone_Emin', 'tone_F#Maj', 
                      'tone_F#min', 'tone_FMaj', 'tone_Fmin', 'tone_G#Maj', 
                      'tone_G#min', 'tone_GMaj', 'tone_Gmin', 
                      'PC1_Energetic_Dynamic', 'PC2_Speech_Tempo', 'PC3_Acoustic_Speech', 
                      'PC4_Duration_Mood ', 'PC5_Acoustic_Liveness']
    # Ensure all expected features are present, adding missing ones as zeros
    input_data = pd.DataFrame(columns=model_features)
    
    for column in tdata.columns:
        if column in model_features:
            input_data[column] = tdata[column]
    
    input_data.fillna(0, inplace=True)

    # Reorder columns to match training
    tdata = tdata[model_features]

    return tdata

In [281]:
def preprocess_data(track_data):
    # Convert track_data into a DataFrame
    tdata = pd.DataFrame(track_data)

    # Processing
    tdata['track_age_2024'] = 2024 - \
        pd.to_numeric(tdata['year'], errors='coerce')
    tdata['dominant_genre'] = tdata['artist_genres'].apply(
        lambda x: categorize_genres(x, flat_mapping))

    key_mapping = {0: 'C', 1: 'C#', 2: 'D', 3: 'D#', 4: 'E',
                   5: 'F', 6: 'F#', 7: 'G', 8: 'G#', 9: 'A', 10: 'A#', 11: 'B'}
    tdata['key'] = tdata['key'].map(key_mapping)
    mode_mapping = {0: 'min', 1: 'Maj'}
    tdata['mode'] = tdata['mode'].map(mode_mapping)
    tdata['tonality'] = tdata['key'] + tdata['mode']

    # One Hot Encoding for 'dominant_genre' and 'tonality'
    tdata = pd.get_dummies(tdata, columns=['dominant_genre', 'tonality'], prefix=[
                           'genre', 'tone'], drop_first=True)

    # Prepare and transform audio features with PCA
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
    pca_components = sklearn_pca_final.transform(tdata[audio_features])
    pca_feature_names = ['PC1_Energetic_Dynamic', 'PC2_Speech_Tempo', 'PC3_Acoustic_Speech', 'PC4_Duration_Mood', 'PC5_Acoustic_Liveness']
    for i, name in enumerate(pca_feature_names):
        tdata[name] = pca_components[:, i]

    # Define expected features as per model's training
    expected_features = ['track_age_2024', 'genre_country', 'genre_folk', 'genre_hip_hop_rap', 'genre_house_and_pop_fusion', 'genre_jazz', 'genre_latin', 'genre_metal', 'genre_other', 'genre_pop', 'genre_reggae', 'genre_rnb_soul', 'genre_rock', 'genre_singer-songwriter', 'genre_underground_electronic', 'tone_A#Maj', 'tone_AMaj', 'tone_BMaj', 'tone_C#Maj', 'tone_CMaj', 'tone_D#Maj', 'tone_DMaj', 'tone_EMaj', 'tone_FMaj', 'tone_G#Maj', 'tone_GMaj', 'PC1_Energetic_Dynamic', 'PC2_Speech_Tempo', 'PC3_Acoustic_Speech', 'PC4_Duration_Mood', 'PC5_Acoustic_Liveness']

    # Ensure all expected features are present, adding missing ones as zeros
    for feature in expected_features:
        if feature not in tdata.columns:
            tdata[feature] = 0

    # Reorder columns to match training
    tdata = tdata[expected_features]

    return tdata

In [282]:
def preprocess_data_final(track_data):
    # Convert track_data into a DataFrame
    tdata = pd.DataFrame(track_data)# Ensure it's a list of dict if track_data is a single dict
    tdata = tdata.head(1)

    # Convert 'year' to 'track_age_2024'
    tdata['track_age_2024'] = 2024 - pd.to_numeric(tdata['year'], errors='coerce')

    # Apply genre categorization
    tdata['dominant_genre'] = tdata['artist_genres'].apply(lambda x: categorize_genres(x, flat_mapping))

    # Map 'key' and 'mode' to their string representations
    key_mapping = {0: 'C', 1: 'C#', 2: 'D', 3: 'D#', 4: 'E', 5: 'F', 6: 'F#', 7: 'G', 8: 'G#', 9: 'A', 10: 'A#', 11: 'B'}
    mode_mapping = {0: 'min', 1: 'Maj'}
    tdata['key'] = tdata['key'].map(key_mapping)
    tdata['mode'] = tdata['mode'].map(mode_mapping)
    tdata['tonality'] = tdata['key'] + "_" + tdata['mode']

    # Drop columns not used in PCA before one-hot encoding to prevent data leakage
    tdata.drop(columns=['year', 'key', 'mode', 'track_name', 'album', 'artist_name', 'artist_genres', 'track_popularity'], inplace=True)

    # One-hot encode categorical features
    tdata = pd.get_dummies(tdata, columns=['dominant_genre', 'tonality'], prefix=['genre', 'tone'], drop_first=True)

    # Prepare audio features for PCA
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
    pca_components = sklearn_pca_final.transform(tdata[audio_features])

    # Add PCA components to DataFrame
    pca_feature_names = ['PC1_Energetic_Dynamic', 'PC2_Speech_Tempo', 'PC3_Acoustic_Speech', 'PC4_Duration_Mood', 'PC5_Acoustic_Liveness']
    for i, name in enumerate(pca_feature_names):
        tdata[name] = pca_components[:, i]

    # Drop original audio features after PCA transformation
    tdata.drop(columns=audio_features, inplace=True)

    # Ensure the order of columns matches the training dataset
    expected_features = ['track_age_2024', 'genre_country', 'genre_folk', 'genre_hip_hop_rap', 'genre_house_and_pop_fusion', 'genre_jazz', 'genre_latin', 'genre_metal', 'genre_other', 'genre_pop', 'genre_reggae', 'genre_rnb_soul', 'genre_rock', 'genre_singer-songwriter', 'genre_underground_electronic'] + [f'tone_{tone}' for tone in ['A#min', 'AMaj', 'Amin', 'BMaj', 'Bmin', 'C#Maj', 'C#min', 'CMaj', 'Cmin', 'D#Maj', 'D#min', 'DMaj', 'Dmin', 'EMaj', 'Emin', 'F#Maj', 'F#min', 'FMaj', 'Fmin', 'G#Maj', 'G#min', 'GMaj', 'Gmin']] + ['PC1_Energetic_Dynamic', 'PC2_Speech_Tempo', 'PC3_Acoustic_Speech', 'PC4_Duration_Mood ', 'PC5_Acoustic_Liveness']

    # Reorder and ensure all expected columns are present
    for feature in expected_features:
        if feature not in tdata.columns:
            tdata[feature] = 0  # Add missing features as zeros
    tdata = tdata[expected_features]

    return tdata


In [283]:
check = preprocess_data_final(track_info_df)



/Users/lucazosso/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


In [284]:
check.head(1)

,track_age_2024,genre_country,genre_folk,genre_hip_hop_rap,genre_house_and_pop_fusion,genre_jazz,genre_latin,genre_metal,genre_other,genre_pop,...,tone_Fmin,tone_G#Maj,tone_G#min,tone_GMaj,tone_Gmin,PC1_Energetic_Dynamic,PC2_Speech_Tempo,PC3_Acoustic_Speech,PC4_Duration_Mood,PC5_Acoustic_Liveness
0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,20991.867471,43436.721565,-54874.040708,0,138698.748359


In [285]:
check.columns

Index(['track_age_2024', 'genre_country', 'genre_folk', 'genre_hip_hop_rap',
       'genre_house_and_pop_fusion', 'genre_jazz', 'genre_latin',
       'genre_metal', 'genre_other', 'genre_pop', 'genre_reggae',
       'genre_rnb_soul', 'genre_rock', 'genre_singer-songwriter',
       'genre_underground_electronic', 'tone_A#min', 'tone_AMaj', 'tone_Amin',
       'tone_BMaj', 'tone_Bmin', 'tone_C#Maj', 'tone_C#min', 'tone_CMaj',
       'tone_Cmin', 'tone_D#Maj', 'tone_D#min', 'tone_DMaj', 'tone_Dmin',
       'tone_EMaj', 'tone_Emin', 'tone_F#Maj', 'tone_F#min', 'tone_FMaj',
       'tone_Fmin', 'tone_G#Maj', 'tone_G#min', 'tone_GMaj', 'tone_Gmin',
       'PC1_Energetic_Dynamic', 'PC2_Speech_Tempo', 'PC3_Acoustic_Speech',
       'PC4_Duration_Mood ', 'PC5_Acoustic_Liveness'],
      dtype='object')

In [286]:
model = load('model.joblib')

In [287]:
def predict_pop(processed_data):
    # Load the trained model
    prediction = model.predict(processed_data)
    return prediction

In [288]:
pred = predict_pop(check)
pred

array([1.])

In [289]:
type(pred)

numpy.ndarray

In [290]:
if pred == 1:
    print("It's a success")
else:
    print("The key of success is in practice, keep working a bit")


It's a success
